### Read scRNA Data (10x)

In [ ]:
import scanpy as sc
import scrublet as scr
import numpy as np
import anndata as ad
import os
import warnings
warnings.filterwarnings("ignore")

# ---------------------------
# Step: Define base directory
# ---------------------------
base_dir = "./GSE163465"

# --------------------
# Step: Define samples
# --------------------
folders = sorted([f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f)) and not f.startswith(".")])
samples = {f"Sample_{i+1}": {"folder": folder} for i, folder in enumerate(folders)}

samples = {
    "Sample_1": {"folder": "GSM4985022"},
    "Sample_2": {"folder": "GSM4985023"},
    "Sample_3": {"folder": "GSM4985024"},
    # "Sample_3": {"folder": "GSM4985024", "group": "WT"},
}

# ----------------------
# Step: Load each sample
# ----------------------
adatas = []

for label, sample_info in samples.items():
    folder = sample_info["folder"]
    sample_dir = os.path.join(base_dir, folder)

    if not os.path.exists(sample_dir):
        raise FileNotFoundError(f"Path does not exist: {sample_dir}")

    adata = sc.read_10x_mtx(
        sample_dir,
        var_names="gene_symbols",
        cache=True
    )
    adata.var_names_make_unique()
    print(f"Loaded {folder}: {adata.shape}")

    # add metadata
    adata.obs_names = [f"{label}_{x}" for x in adata.obs_names]
    adata.obs["batch"] = label
    # adata.obs["group"] = sample_info.get("group", "NA")
    # order = ['WT', 'KO', 'KI']
    # adata.obs['group'] = pd.Categorical(adata_sub.obs['group'], categories=order, ordered=True)

    # save raw counts for future use
    adata.raw = adata.copy()
    adatas.append(adata)

# -----------------
# Step: Concatenate
# -----------------
adata = ad.concat(adatas, join="outer", label="sample", keys=samples.keys())
print(adata)

### Read scRNA Data (.h5ad)

In [ ]:
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

adata = sc.read_h5ad("GSE243170_an_geo_2023.h5ad")

### Quality Control

In [ ]:
# ------------------------
# Step: Compute QC Metrics
# ------------------------
# Identify mitochondrial genes (human: MT-, mouse: mt-)
adata.var["MT"] = adata.var_names.str.startswith(("MT-", "mt-"))

sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=["MT"],
    percent_top=None,
    log1p=False,
    inplace=True
)

# ------------------------------------
# Step: Plot QC Violin & Scatter Plots
# ------------------------------------
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_MT"],
    jitter=0.4,
    multi_panel=True
)
sc.pl.scatter(adata, x="total_counts", y="pct_counts_MT")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

In [ ]:
import scrublet as scr

# --------------------------------
# Step: Scrublet doublet detection
# --------------------------------
counts_matrix = adata.X.copy()  # sparse CSR matrix

scrub = scr.Scrublet(counts_matrix)
doublet_scores, predicted_doublets_auto = scrub.scrub_doublets(min_counts=2)
predicted_doublets_manual = scrub.call_doublets(threshold=0.25)

if predicted_doublets_auto is not None and len(predicted_doublets_auto) == adata.n_obs:
    predicted_doublets = predicted_doublets_auto
    print("Using auto-predicted doublets.")
else:
    predicted_doublets = predicted_doublets_manual
    print("Auto prediction not available; using manual threshold.")

adata.obs['doublet_score'] = doublet_scores
adata.obs['predicted_doublet'] = predicted_doublets

print(adata.obs['predicted_doublet'].value_counts())

In [ ]:
# ----------------------------------
# Step: Gene and cell filtering (QC)
# ----------------------------------
# Remove genes expressed in fewer than 3 cells
sc.pp.filter_genes(adata, min_cells=3)

# Remove cells expressing fewer than 200 genes or more than 5000 genes
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_cells(adata, max_genes=5000)

# Remove cells with high mitochondrial gene content (>5%)
adata = adata[adata.obs.pct_counts_MT < 5, :].copy()

# -------------------------------
# Step: Remove predicted doublets
# -------------------------------
adata = adata[adata.obs['predicted_doublet'] == False, :].copy()

# ------------------------------
# Step: Save raw counts after QC
# ------------------------------
adata.raw = adata.copy()
adata.write("adata_raw_count_afterQC.h5ad", compression='gzip')
print("Save adata after QC. *adata.raw contains raw counts")

# --------------------
# Step: Normalization
# --------------------
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Save the normalized data as raw for downstream analysis
adata.raw = adata.copy()

# -------------------------
# Step: Backup post-QC data
# -------------------------
adata_qc = adata.copy()
print(f"Shape after QC: {adata_qc.shape}")